In [35]:
import json
import elasticsearch
from dotenv import load_dotenv
import os
load_dotenv()

True

In [36]:
import os
os.getcwd()

'D:\\Projects\\AI-Restaurent-Chat-bot\\notebooks'

In [37]:
# change path for base folder
base_folder = 'D:/Projects/AI-Restaurent-Chat-bot/'
input_data_folder = base_folder+'input_data/'

In [38]:
# openai api
from openai import OpenAI
client = OpenAI()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [39]:
with open(input_data_folder + 'food_user_qa_dataset.json', 'rt') as f_in:
    data = json.load(f_in)

In [41]:
documents = []
for dish in data['dishes']:
    dish_name = dish['dish name']
    for doc in dish['documents']:
        doc['dish_name'] = dish_name  # Add dish_name to each document
        documents.append(doc)

In [87]:
documents[1:5]

[{'id': '142_2',
  'question': 'How many calories does it have?',
  'section': 'calories',
  'text': 'The almond fudge banana cake has 224.8 calories.',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_3',
  'question': 'How much total fat does it contain?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has 14 grams of total fat (PDV).',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_4',
  'question': 'What is the sugar content?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has 87 grams of sugar (PDV).',
  'dish_name': 'almond fudge banana cake'},
 {'id': '142_5',
  'question': 'How much sodium is in this dish?',
  'section': 'nutritional',
  'text': 'The almond fudge banana cake has 10 grams of sodium (PDV).',
  'dish_name': 'almond fudge banana cake'}]

In [43]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '81d6d96573fa', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Knoc8xY-Teq-lYkTwu4Eww', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [44]:

index_name = "menu-items"

# Check if index exists
if not es.indices.exists(index=index_name):
    # Create the index with the correct mapping
    response = es.indices.create(
        index=index_name,
        body={
            "settings": {
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
            "mappings": {
                "properties": {
                    "category": {"type": "text"},
                    "dishes": {
                        "type": "nested",
                        "properties": {
                            "dish name": {"type": "text"},
                            "documents": {
                                "type": "nested",
                                "properties": {
                                    "id": {"type": "keyword"},
                                    "question": {"type": "text"},
                                    "section": {"type": "text"},
                                    "text": {"type": "text"}
                                }
                            }
                        }
                    }
                }
            }
        }
    )
    print("Index creation response:", response)
else:
    print(f"Index '{index_name}' already exists.")


Index 'menu-items' already exists.


In [88]:

index_name = "menu-items"

# Check if index exists
if not es.indices.exists(index=index_name):
    # Create the index with the correct mapping
    response = es.indices.create(
        index=index_name,
        body={
            "settings": {
                "number_of_shards": 1,
                "number_of_replicas": 0
            },
            "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "dish_name": {"type": "keyword"},
            "id": {"type": "keyword"}
        }
    }
        }
    )
    print("Index creation response:", response)
else:
    print(f"Index '{index_name}' already exists.")


Index 'menu-items' already exists.


In [91]:
from tqdm.auto import tqdm
from elasticsearch.exceptions import NotFoundError, RequestError

for doc in tqdm(documents):
    try:
        es.index(index=index_name, document=doc)
    except (NotFoundError, RequestError) as e:
        print(f"Error indexing document: {doc}")
        print(e)


100%|██████████████████████████████████████████████████████████| 3584/3584 [03:17<00:00, 18.18it/s]


In [92]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
index_name = "menu-items"

# Example search query
query = {
    "query": {
        "match": {
            "documents.text": "banana"
        }
    }
}

response = es.search(index=index_name, body=query)
print(response)


{'took': 1275, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 0, 'relation': 'eq'}, 'max_score': None, 'hits': []}}


In [93]:
response = es.search(index=index_name, body={"query": {"match_all": {}}})
print(response)

{'took': 33, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 10000, 'relation': 'gte'}, 'max_score': 1.0, 'hits': [{'_index': 'menu-items', '_id': 'q1NncJEBM82uNR2Or_9y', '_score': 1.0, '_source': {'id': '3091.0_1', 'question': 'What are the ingredients?', 'section': 'ingredients', 'text': "The kitty litter cake contains the following ingredients: ['spice cake mix', 'eggs', 'oil', 'water', 'white cake mix', 'vanilla instant pudding mix', 'cookies', 'green food coloring', 'tootsie rolls', 'new kitty litter box', 'pooper scooper']", 'dish_name': 'kitty litter cake'}}, {'_index': 'menu-items', '_id': 'rFNncJEBM82uNR2Or_-u', '_score': 1.0, '_source': {'id': '3091.0_2', 'question': 'How many calories does it have?', 'section': 'calories', 'text': 'The kitty litter cake has 354.2 calories.', 'dish_name': 'kitty litter cake'}}, {'_index': 'menu-items', '_id': 'rVNncJEBM82uNR2Or__h', '_score': 1.0, '_source': {'id': '3091.0_3

In [46]:
# Check index mapping
mapping = es.indices.get_mapping(index=index_name)
print(mapping)

# Check index settings
settings = es.indices.get_settings(index=index_name)
print(settings)

{'menu-items': {'mappings': {'properties': {'dish_name': {'type': 'keyword'}, 'id': {'type': 'keyword'}, 'question': {'type': 'text'}, 'section': {'type': 'text'}, 'text': {'type': 'text'}}}}}
{'menu-items': {'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}}, 'number_of_shards': '1', 'provided_name': 'menu-items', 'creation_date': '1724167714048', 'number_of_replicas': '0', 'uuid': 'LYmVzOZIRcqRaTQhnwAfZQ', 'version': {'created': '8040399'}}}}}


In [94]:
query = "almond fudge banana cake"

In [98]:
documents[0]

{'id': '142_1',
 'question': 'What are the ingredients?',
 'section': 'ingredients',
 'text': "The almond fudge banana cake contains the following ingredients: ['dole banana', 'sugar', 'margarine', 'eggs', 'amaretto liqueur', 'vanilla extract', 'all-purpose flour', 'unsweetened cocoa powder', 'baking soda', 'salt', 'dole almond']",
 'dish_name': 'almond fudge banana cake'}

In [116]:
def elastic_search(query, dish_name=None):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": [
                    {
                        "multi_match": {
                            "query": query,
                            "fields": ["dish_name^3", "question^3", "text", "section"],
                            "type": "best_fields"
                        }
                    }
                ]
            }
        }
    }

    # Add a filter by dish_name if it's provided
    if dish_name:
        search_query["query"]["bool"]["filter"] = {
            "term": {
                "dish_name.keyword": dish_name
            }
        }

    # Perform the search
    response = es.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        doc = hit['_source']
        result_docs.append(doc)  # Collect all documents
        

    return result_docs


In [135]:
elastic_search(query)

[{'id': '3091.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The kitty litter cake contains the following ingredients: ['spice cake mix', 'eggs', 'oil', 'water', 'white cake mix', 'vanilla instant pudding mix', 'cookies', 'green food coloring', 'tootsie rolls', 'new kitty litter box', 'pooper scooper']",
  'dish_name': 'kitty litter cake'},
 {'id': '3333.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The dirt cake ii contains the following ingredients: ['oreo cookies', 'butter', 'cream cheese', 'instant vanilla pudding', 'milk', 'cool whip', 'flower pot', 'artificial flower']",
  'dish_name': 'dirt cake ii'},
 {'id': '822.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The heloise s cake mix cookies contains the following ingredients: ['cake mix', 'vegetable oil', 'eggs']",
  'dish_name': 'heloise s cake mix cookies'},
 {'id': '520.0_1',
  'question': 'What are the 

In [132]:
def build_prompt(query, search_results):
    prompt_template = """
You're an AI assistant helping with menu queries. Answer the QUESTION based on the CONTEXT provided.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context += f"Section: {doc['section']}\nQuestion: {doc['question']}\nAnswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt


In [133]:
query

'what are ingredients in banana pudding cake contain'

In [134]:
search_results= elastic_search(query)
search_results

[{'id': '3091.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The kitty litter cake contains the following ingredients: ['spice cake mix', 'eggs', 'oil', 'water', 'white cake mix', 'vanilla instant pudding mix', 'cookies', 'green food coloring', 'tootsie rolls', 'new kitty litter box', 'pooper scooper']",
  'dish_name': 'kitty litter cake'},
 {'id': '3333.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The dirt cake ii contains the following ingredients: ['oreo cookies', 'butter', 'cream cheese', 'instant vanilla pudding', 'milk', 'cool whip', 'flower pot', 'artificial flower']",
  'dish_name': 'dirt cake ii'},
 {'id': '822.0_1',
  'question': 'What are the ingredients?',
  'section': 'ingredients',
  'text': "The heloise s cake mix cookies contains the following ingredients: ['cake mix', 'vegetable oil', 'eggs']",
  'dish_name': 'heloise s cake mix cookies'},
 {'id': '520.0_1',
  'question': 'What are the 

In [119]:
# if using OPENAI 
from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [120]:
# iF USING groq
"""
from groq import Groq   

#create client calling Groq class

client = Groq(api_key=os.getenv('GROQ_API_KEY'))

from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

"""

'\nfrom groq import Groq   \n\n#create client calling Groq class\n\nclient = Groq(api_key=os.getenv(\'GROQ_API_KEY\'))\n\nfrom openai import OpenAI\nclient = OpenAI()\n\ndef llm(prompt):\n    response = client.chat.completions.create(\n        model="llama3-8b-8192",\n        messages=[{"role": "user", "content": prompt}]\n    )\n    \n    return response.choices[0].message.content\n\n'

In [150]:
query = 'dish name please'

def rag(query):
    search_results = elastic_search(query)
    prompt=build_prompt(query,search_results)
    answer = llm(prompt)
    return answer

In [151]:
rag(query)


"I'm sorry, the dish name you are asking for is baby food pineapple coconut carrot cake."